In [2]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

from newrow import newrow_cash, newrow_etf_stock
from utils import broker_fee, get_date, round_half_up, get_asset_value
from fetch_data import fetch_exchange_rate
import requests
from secret import OXCR_KEY
from datetime import date, timedelta, datetime
import json

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [6]:
dati_utente = {
}

idx = 1
while True:
    new_broker = input("    Inserisci nuovo broker > ")
    if new_broker == "q":
        break
    dati_utente[idx] = new_broker
    print(f"{idx}. {new_broker}")
    idx += 1
    

In [7]:
print(len(dati_utente))

0
